In [2]:
import numpy as np
import math
import pennylane as qml
from pennylane.operation import Operation, AnyWires
# Importing standard Qiskit libraries
import qiskit as qis
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator

# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()

ibmqfactory.load_account:WARNING:2022-10-18 14:51:19,143: Credentials are already in use. The existing account in the session will be replaced.


In [3]:
def graycode(rank):
    def gray_code_recurse(g, rank):
        k = len(g)
        if rank <= 0:
            return
        for i in range(k - 1, -1, -1):
            char = "1" + g[i]
            g.append(char)
        for i in range(k - 1, -1, -1):
            g[i] = "0" + g[i]
        gray_code_recurse(g, rank - 1)
    g = ["0", "1"]
    gray_code_recurse(g, rank - 1)
    return g

In [4]:
def gray(n):
    n ^= (n >> 1)
    return bin(n)[2:]

def dotP(i,j):
    binary = np.unpackbits(np.array([int(np.binary_repr(i),2)],dtype=np.uint8))
    gcode = np.unpackbits(np.array([int(gray(j),2)],dtype=np.uint8))
    sum1s = np.dot(binary,gcode)
    return sum1s

In [5]:
def compute_theta(alpha):
    k = np.log2(len(alpha))
    N = int(2**k)
    M = np.empty((N,N))
    for i in range(N):
        for j in range(N):
            M[i,j] = (1/2**(k))*((-1)**dotP(j,i))
    theta = np.matmul(M,alpha)
    return theta

In [6]:
def alpha_z(omega, n, k):
    alpha = []
    for j in range(2**(n-k)):
        angle = 0
        for l in range(2**(k-1)):
            angle = angle + (omega[(2*(j+1)-1)*(2**(k-1))+(l)] - omega[(2*(j+1)-2)*(2**(k-1))+(l)])/(2**(k-1))
        alpha.append(angle)
    return alpha

In [7]:
def alpha_y(a, n, k):
    alpha = []
    for j in range(2**(n-k)):
        num = 0
        denom = 0
        for l in range(2**(k-1)):
            num = num + abs(a[(2*(j+1)-1)*(2**(k-1))+l])**2
        for l in range(2**k):
            denom = denom + abs(a[j*(2**k)+l])**2
        angle = 2*(math.asin(np.sqrt(num/denom)))
        alpha.append(angle)
    return alpha

In [8]:
def _apply_uniform_rotation_dagger(gate, alpha, control_wires, target_wire):
    op_list = []
    theta = compute_theta(alpha)

    gray_code_rank = len(control_wires)

    if gray_code_rank == 0:
        if qml.math.all(theta[..., 0] != 0.0):
            op_list.append(gate(theta[..., 0], wires=[target_wire]))
        return op_list

    code = gray_code(gray_code_rank)
    num_selections = len(code)

    control_indices = [
        int(np.log2(int(code[i], 2) ^ int(code[(i + 1) % num_selections], 2)))
        for i in range(num_selections)
    ]

    for i, control_index in enumerate(control_indices):
        if qml.math.all(theta[..., i] != 0.0):
            op_list.append(gate(theta[..., i], wires=[target_wire]))
        op_list.append(qml.CNOT(wires=[control_wires[control_index], target_wire]))
    return op_list

In [9]:
class MottonenStatePreparation(Operation):
    num_wires = AnyWires
    grad_method = None

    def __init__(self, state_vector, wires, do_queue=True, id=None):

        # check if the `state_vector` param is batched
        batched = len(qml.math.shape(state_vector)) > 1

        state_batch = state_vector if batched else [state_vector]

        # apply checks to each state vector in the batch
        for i, state in enumerate(state_batch):
            shape = qml.math.shape(state)

            if len(shape) != 1:
                raise ValueError(
                    f"State vectors must be one-dimensional; vector {i} has shape {shape}."
                )

            n_amplitudes = shape[0]
            if n_amplitudes != 2 ** len(qml.wires.Wires(wires)):
                raise ValueError(
                    f"State vectors must be of length {2 ** len(wires)} or less; vector {i} has length {n_amplitudes}."
                )

            norm = qml.math.sum(qml.math.abs(state) ** 2)
            if not qml.math.allclose(norm, 1.0, atol=1e-3):
                raise ValueError(
                    f"State vectors have to be of norm 1.0, vector {i} has norm {norm}"
                )

        super().__init__(state_vector, wires=wires, do_queue=do_queue, id=id)

    def num_params(self):
        return 1
    
    def compute_decomposition(state_vector, wires):  # pylint: disable=arguments-differ
        a = qml.math.abs(state_vector)
        omega = qml.math.angle(state_vector)
        # change ordering of wires, since original code
        # was written for IBM machines
        wires_reverse = wires[::-1]

        op_list = []

        # Apply inverse y rotation cascade to prepare correct absolute values of amplitudes
        for k in range(len(wires_reverse), 0, -1):
            alpha_y_k = _get_alpha_y(a, len(wires_reverse), k)
            control = wires_reverse[k:]
            target = wires_reverse[k - 1]
            op_list.extend(_apply_uniform_rotation_dagger(qml.RY, alpha_y_k, control, target))

        # If necessary, apply inverse z rotation cascade to prepare correct phases of amplitudes
        if not qml.math.allclose(omega, 0):
            for k in range(len(wires_reverse), 0, -1):
                alpha_z_k = _get_alpha_z(omega, len(wires_reverse), k)
                control = wires_reverse[k:]
                target = wires_reverse[k - 1]
                if len(alpha_z_k) > 0:
                    op_list.extend(
                        _apply_uniform_rotation_dagger(qml.RZ, alpha_z_k, control, target)
                    )

        return op_list


In [10]:
dev = qml.device('default.qubit', wires=3)

@qml.qnode(dev)
def circuit(state):
    qml.MottonenStatePreparation(state_vector=state, wires=range(3))
    return qml.state()

state = np.array([1, 2j, -1, 7j, 5, 6j, 8, 5j])
state = state / np.linalg.norm(state)

print(qml.draw(circuit, expansion_strategy="device", max_length=80)(state))

0: ──RY(2.05)─╭●───────────╭●───────────────╭●─────────────────────────╭●
1: ──RY(2.14)─╰X──RY(0.39)─╰X─╭●────────────│─────────────╭●───────────│─
2: ──RY(1.99)─────────────────╰X──RY(-0.00)─╰X──RY(-0.32)─╰X──RY(0.55)─╰X

───RZ(-0.79)─╭●───────────╭●──────────────╭●─────────────────────────╭●─┤  State
───RZ(0.79)──╰X──RZ(0.79)─╰X─╭●───────────│────────────╭●────────────│──┤  State
───RZ(0.79)──────────────────╰X──RZ(0.79)─╰X──RZ(0.79)─╰X──RZ(-0.79)─╰X─┤  State
